In [1]:
# Load required libraries
library(Seurat)
library(dplyr)
library(ggplot2)
library(patchwork)
library(Matrix)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
samples <- list(
  HC1 = list(
    barcodes = "GSE266852_OES19441150_barcodes.tsv",
    features = "GSE266852_OES19441150_features.tsv",
    matrix = "GSE266852_OES19441150_matrix.mtx",
    condition = "Healthy",
    sample_id = "HC_01"
  ),
  HC2 = list(
    barcodes = "GSE266852_OES19442150_barcodes.tsv",
    features = "GSE266852_OES19442150_features.tsv",
    matrix = "GSE266852_OES19442150_matrix.mtx",
    condition = "Healthy",
    sample_id = "HC_02"
  ),
  HC3 = list(
    barcodes = "GSE266852_OES19443150_barcodes.tsv",
    features = "GSE266852_OES19443150_features.tsv",
    matrix = "GSE266852_OES19443150_matrix.mtx",
    condition = "Healthy",
    sample_id = "HC_03"
  )
  # Note: Add RA and SLE samples when file names are confirmed
)

In [3]:
s <- readMM(file = 'GSE266852_OES19441150_matrix.mtx')

In [4]:
head(s)

6 x 8899 sparse Matrix of class "dgTMatrix"
                                                                              
[1,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[2,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[3,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[4,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[5,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[6,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
                                                                              
[1,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[2,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[3,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[4,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[5,] . .

## Big picture: what the below code is doing 🧠

This code:

1. **Loads one scRNA-seq sample** (10X-style data: matrix, barcodes, features)
2. **Fixes duplicate gene names** (important for Seurat)
3. **Creates a Seurat object**
4. **Adds metadata**
5. **Repeats this for multiple samples**

So this function = *“Turn raw 10X files into a Seurat object”*.

---

# 1️⃣ What is a function in R?

```r
load_sample <- function(sample_info) {
  ...
}
```

Think of a function as:

> “A reusable mini-program that takes input and returns output”

Here:

* **Function name** → `load_sample`
* **Input** → `sample_info`
* **Output** → a **Seurat object**

You can later do:

```r
load_sample(samples[[1]])
```

---

# 2️⃣ What is `sample_info` and why `$` is used?

### `$` = “take a column / element by name”

In R, `$` is used when you have:

* a **list**
* a **data frame**

Example:

```r
sample_info$matrix
```

means:

> “From `sample_info`, give me the thing called `matrix`”

So `sample_info` probably looks like this:

```r
sample_info = list(
  matrix = "matrix.mtx.gz",
  barcodes = "barcodes.tsv.gz",
  features = "features.tsv.gz",
  sample_id = "CTRL_1",
  condition = "control"
)
```

| Expression              | Meaning              |
| ----------------------- | -------------------- |
| `sample_info$matrix`    | path to matrix file  |
| `sample_info$barcodes`  | path to barcode file |
| `sample_info$features`  | path to gene file    |
| `sample_info$sample_id` | sample name          |
| `sample_info$condition` | control / treated    |

💡 `$` is **not math** — it’s just **“extract by name”**

---

# 3️⃣ Reading the matrix (counts)

```r
counts <- readMM(sample_info$matrix)
```

* `readMM()` reads a **Matrix Market (.mtx)** file
* Output is a **sparse matrix**
* Rows = genes
* Columns = cells

This is the **raw gene expression count matrix**

---

# 4️⃣ Reading barcodes and features

```r
barcodes <- read.table(sample_info$barcodes, header = FALSE)
features <- read.table(sample_info$features, header = FALSE, sep = "\t")
```

### Why `header = FALSE`?

Because these files look like:

```
AAACCTGAGAAACCAT-1
AAACCTGAGAAACGTT-1
```

No column names → so R auto-names columns as:

```
V1, V2, V3 ...
```

### That’s where `V1`, `V2` come from 🔑

| File           | Column | Meaning            |
| -------------- | ------ | ------------------ |
| `barcodes.tsv` | `V1`   | Cell barcode       |
| `features.tsv` | `V1`   | Gene ID (ENSG...)  |
| `features.tsv` | `V2`   | Gene symbol (TP53) |
| `features.tsv` | `V3`   | Feature type       |

So:

```r
features$V2
```

means:

> “Second column of features file = gene names”

---

# 5️⃣ Why duplicate gene names are a problem

```r
gene_names <- features$V2
```

Example:

```
MALAT1
MALAT1
RPL13
RPL13
```

🚨 **Seurat does NOT allow duplicated row names**

So we fix them.

---

## Option 1: Make them unique

```r
gene_names <- make.unique(as.character(gene_names))
```

This converts:

```
MALAT1
MALAT1
```

into:

```
MALAT1
MALAT1.1
```

✔ Keeps gene symbols
✔ Safe for Seurat

---

## Option 2 (commented): Use gene IDs instead

```r
# if (any(duplicated(features$V2))) {
#   gene_names <- features$V1
# }
```

* Uses **ENSEMBL IDs**
* 100% unique
* Less human-readable

You commented this out, so **Option 1 is used**

---

# 6️⃣ Assigning row and column names

```r
rownames(counts) <- gene_names
colnames(counts) <- barcodes$V1
```

This means:

| Matrix part     | Assigned from |
| --------------- | ------------- |
| rows (genes)    | gene names    |
| columns (cells) | barcodes      |

So now your matrix looks like:

```
          Cell1   Cell2   Cell3
TP53        0       1       0
MALAT1     12      15       9
```

---

# 7️⃣ Creating the Seurat object 🧬

```r
seurat_obj <- CreateSeuratObject(
  counts = counts,
  project = sample_info$sample_id,
  min.cells = 3,
  min.features = 200
)
```

### What this does biologically:

* Removes genes expressed in < 3 cells
* Removes cells expressing < 200 genes

### Technically:

* Wraps your matrix into a **Seurat class**
* Adds slots like:

  * `@assays`
  * `@meta.data`
  * `@reductions`

---

# 8️⃣ Adding metadata (this is important!)

```r
seurat_obj$condition <- sample_info$condition
seurat_obj$sample_id <- sample_info$sample_id
```

Here `$` again means:

> “Add a column to Seurat metadata”

Now every cell has:

| cell   | condition | sample_id |
| ------ | --------- | --------- |
| cell_1 | control   | CTRL_1    |
| cell_2 | control   | CTRL_1    |

This is **CRUCIAL** later for:

* Differential expression
* Integration
* Batch correction

---

# 9️⃣ Returning the object

```r
return(seurat_obj)
```

Means:

> “Give this Seurat object back to whoever called the function”

---

# 🔟 Loading all samples

```r
seurat_list <- lapply(samples, load_sample)
```

### `lapply()` in plain English:

> “For each element in `samples`, run `load_sample()`”

If `samples` has 3 entries:

* `samples[[1]]`
* `samples[[2]]`
* `samples[[3]]`

Then output:

```r
seurat_list = list(
  seurat_obj_1,
  seurat_obj_2,
  seurat_obj_3
)
```

---

# 1️⃣1️⃣ Checking results

```r
sapply(seurat_list, function(x) dim(x))
```

* `dim(x)` → number of genes × cells
* `sapply()` → applies this to each Seurat object

Output like:

```
        sample1 sample2 sample3
genes     18000   17500   17800
cells      4500    4200    4600
```

In [5]:
# Modified load_sample function with duplicate handling
load_sample <- function(sample_info) {
  # Read the matrix market file
  counts <- readMM(sample_info$matrix)
  
  # Read barcodes and features
  barcodes <- read.table(sample_info$barcodes, header = FALSE)
  features <- read.table(sample_info$features, header = FALSE, sep = "\t")
  
  # Check for and handle duplicate gene names
  gene_names <- features$V2
  
  # Option 1: Make unique by appending numbers to duplicates
  gene_names <- make.unique(as.character(gene_names))
  
  # OR Option 2: Use Gene IDs (V1) if duplicates exist
  # if (any(duplicated(features$V2))) {
  #   message("Duplicate gene names found in ", sample_info$sample_id, 
  #           ". Using gene IDs instead.")
  #   gene_names <- features$V1
  # }
  
  # Set matrix dimensions
  rownames(counts) <- gene_names  # Unique gene names
  colnames(counts) <- barcodes$V1  # Cell barcodes
  
  # Create Seurat object
  seurat_obj <- CreateSeuratObject(
    counts = counts,
    project = sample_info$sample_id,
    min.cells = 3,
    min.features = 200
  )
  
  # Add metadata
  seurat_obj$condition <- sample_info$condition
  seurat_obj$sample_id <- sample_info$sample_id
  
  return(seurat_obj)
}


In [6]:
# Load all samples
print("Loading data from all samples...")
seurat_list <- lapply(samples, load_sample)

[1] "Loading data from all samples..."


Warning message:
“Data is of class dgTMatrix. Coercing to dgCMatrix.”
Warning message:
“Data is of class dgTMatrix. Coercing to dgCMatrix.”
Warning message:
“Data is of class dgTMatrix. Coercing to dgCMatrix.”


In [7]:
# Check loading success
print(paste("Loaded", length(seurat_list), "samples successfully"))
sapply(seurat_list, function(x) dim(x))

[1] "Loaded 3 samples successfully"


HC1,HC2,HC3
18682,18505,18501
8865,7684,7667


In [8]:
hc1 <- seurat_list$HC1
head(colnames(hc1), 5)

[1] "AAACCCAAGCTTACGT-1" "AAACCCACAATTCACG-1" "AAACCCACACGCACCA-1"
[4] "AAACCCACATCTGGGC-1" "AAACCCAGTAAGGCCA-1"

In [9]:
counts_hc1 <- GetAssayData(hc1, assay='RNA', layer='counts')
print(counts_hc1[1:5, 1:5])

5 x 5 sparse Matrix of class "dgCMatrix"
           AAACCCAAGCTTACGT-1 AAACCCACAATTCACG-1 AAACCCACACGCACCA-1
AL627309.1                  .                  .                  .
AL669831.2                  .                  .                  .
AL669831.5                  .                  .                  .
FAM87B                      .                  .                  .
LINC00115                   .                  .                  .
           AAACCCACATCTGGGC-1 AAACCCAGTAAGGCCA-1
AL627309.1                  .                  .
AL669831.2                  .                  .
AL669831.5                  .                  .
FAM87B                      .                  .
LINC00115                   .                  .


In [10]:
cat("Sparsity of HC1:", round(1 - nnzero(counts_hc1) / length(counts_hc1), 4)*100, "% zeros\n")

Sparsity of HC1: 91.84 % zeros


## Big picture first 🧠

This block:

👉 **adds QC (quality control) metrics** to **each Seurat object** in `seurat_list`

Specifically, **per cell** it calculates:

* % of mitochondrial genes
* % of ribosomal genes (optional)

These values are later used to **filter bad cells**.

---

# 1️⃣ What is `seurat_list` here?

From before:

```r
seurat_list <- lapply(samples, load_sample)
```

So:

* `seurat_list` is a **list**
* Each element = **one Seurat object**
* Named by sample IDs

Example mental model:

```r
seurat_list = list(
  CTRL_1 = <Seurat object>,
  CTRL_2 = <Seurat object>,
  TRT_1  = <Seurat object>
)
```

---

# 2️⃣ Understanding the `for` loop

```r
for (i in names(seurat_list)) {
  ...
}
```

### What is `names(seurat_list)`?

```r
names(seurat_list)
# [1] "CTRL_1" "CTRL_2" "TRT_1"
```

So the loop does:

```
i = "CTRL_1"
i = "CTRL_2"
i = "TRT_1"
```

Each time, it processes **one sample**.

---

# 3️⃣ What does `[[i]]` mean?

```r
seurat_list[[i]]
```

This means:

> “Give me the Seurat object named `i` from the list”

⚠️ Important difference:

| Syntax             | Meaning                              |
| ------------------ | ------------------------------------ |
| `seurat_list[i]`   | returns a **list**                   |
| `seurat_list[[i]]` | returns the **actual object inside** |

You **must** use `[[ ]]` here, or Seurat functions won’t work.

---

# 4️⃣ What is `PercentageFeatureSet()`?

```r
PercentageFeatureSet(object, pattern = "...")
```

This function:

1. Finds genes whose names **match a pattern**
2. Calculates:

   ```
   (sum of counts of those genes / total counts per cell) × 100
   ```
3. Returns **one number per cell**

So output length = number of cells.

---

# 5️⃣ Mitochondrial percentage

```r
seurat_list[[i]][["percent.mt"]] <- PercentageFeatureSet(
  seurat_list[[i]], 
  pattern = "^MT-"
)
```

### Let’s break this into pieces 👇

---

## 🔹 `pattern = "^MT-"`

This is a **regular expression (regex)**.

| Symbol | Meaning            |
| ------ | ------------------ |
| `^`    | start of gene name |
| `MT-`  | literal text       |

So this matches genes like:

```
MT-CO1
MT-ND1
MT-ATP6
```

👉 Standard mitochondrial gene naming (human)

---

## 🔹 What does `[["percent.mt"]] <-` do?

This is **Seurat-specific magic**.

```r
seurat_obj[["percent.mt"]] <- values
```

means:

> “Add a new metadata column called `percent.mt`”

Equivalent to:

```r
seurat_obj@meta.data$percent.mt <- values
```

But the `[[ ]]` way is cleaner and safer.

---

## 🔹 What does `percent.mt` represent?

For each **cell**:

```
percent.mt = (mitochondrial counts / total counts) × 100
```

Biologically:

* High `% MT` → damaged / dying cell
* Low `% MT` → healthy cell

Typical cutoff:

```
< 5–10%
```

---

# 6️⃣ Ribosomal percentage (optional)

```r
seurat_list[[i]][["percent.ribo"]] <- PercentageFeatureSet(
  seurat_list[[i]], 
  pattern = "^RP[SL]"
)
```

### Regex explanation:

| Pattern | Matches        |
| ------- | -------------- |
| `^RP`   | starts with RP |
| `[SL]`  | either S or L  |

So this matches:

```
RPS3
RPS12
RPL5
RPL13
```

👉 Ribosomal protein genes

---

## Why ribosomal percentage?

* High ribosomal content → often **stress or technical bias**
* Not always filtered, but useful for diagnostics

---

# 7️⃣ What is happening after the loop?

After the loop finishes:

Each Seurat object now has **extra metadata columns**:

```r
head(seurat_list[[1]]@meta.data)
```

| cell | nFeature_RNA | nCount_RNA | percent.mt | percent.ribo |
| ---- | ------------ | ---------- | ---------- | ------------ |

These are used in:

```r
VlnPlot(seurat_obj, features = c("nFeature_RNA", "percent.mt"))
```

or filtering:

```r
subset(
  seurat_obj,
  subset = percent.mt < 10 & nFeature_RNA > 300
)
```

---

# 8️⃣ Why use a `for` loop instead of `lapply()`?

Because:

```r
seurat_list[[i]][["percent.mt"]] <- ...
```

**modifies objects in place**

`lapply()` returns new objects, which is awkward for metadata assignment.

So `for` loop = correct choice here 👍

---

# 9️⃣ Beginner R summary cheat-sheet 🧾

| Syntax             | Meaning                |
| ------------------ | ---------------------- |
| `for (i in x)`     | loop over x            |
| `names(list)`      | names of list elements |
| `[[ ]]`            | extract actual object  |
| `[["new_col"]] <-` | add metadata column    |
| `^` in regex       | starts with            |
| `[SL]`             | S or L                 |

---


In [11]:
# Calculate QC metrics for each sample
for (i in names(seurat_list)) {
  # Calculate mitochondrial percentage
  seurat_list[[i]][["percent.mt"]] <- PercentageFeatureSet(
    seurat_list[[i]], 
    pattern = "^MT-"
  )
  
  # Calculate ribosomal percentage (optional)
  seurat_list[[i]][["percent.ribo"]] <- PercentageFeatureSet(
    seurat_list[[i]], 
    pattern = "^RP[SL]"
  )
}

In [12]:
# Visualize QC metrics before filtering
pdf("results/QC/01_QC_before_filtering.pdf", width = 15, height = 10)

for (i in names(seurat_list)) {
  p1 <- VlnPlot(
    seurat_list[[i]], 
    features = c("nFeature_RNA", "nCount_RNA", "percent.mt"),
    ncol = 3,
    pt.size = 0.1
  ) + plot_annotation(title = paste("Sample:", i))
  print(p1)
}

dev.off()

Warning message:
“Default search for "data" layer in "RNA" assay yielded no results; utilizing "counts" layer instead.”
Warning message:
“Default search for "data" layer in "RNA" assay yielded no results; utilizing "counts" layer instead.”
Warning message:
“Default search for "data" layer in "RNA" assay yielded no results; utilizing "counts" layer instead.”


agg_record_227801569 
                   2

In [13]:
head(seurat_list[[2]]@meta.data)


,orig.ident,nCount_RNA,nFeature_RNA,condition,sample_id,percent.mt,percent.ribo
,<fct>,<dbl>,<int>,<chr>,<chr>,<dbl>,<dbl>
AAACCCAAGCCAGACA-1,HC_02,11539,2987,Healthy,HC_02,6.725019,28.89332
AAACCCACAACTCCAA-1,HC_02,6033,2174,Healthy,HC_02,5.022377,15.56440
AAACCCACATGCCGGT-1,HC_02,6370,1913,Healthy,HC_02,4.065934,36.87598
AAACCCAGTACTAAGA-1,HC_02,1571,814,Healthy,HC_02,14.704010,10.88479
AAACCCAGTTTATGCG-1,HC_02,4038,1502,Healthy,HC_02,8.172363,26.15156
AAACCCATCACCGGTG-1,HC_02,6846,2026,Healthy,HC_02,3.827052,34.94011


In [14]:
names(seurat_list)

[1] "HC1" "HC2" "HC3"

In [16]:
seurat_list[[1]]

An object of class Seurat 
18682 features across 8865 samples within 1 assay 
Active assay: RNA (18682 features, 0 variable features)
 1 layer present: counts

In [17]:
head(seurat_list[[1]]@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,condition,sample_id,percent.mt,percent.ribo
,<fct>,<dbl>,<int>,<chr>,<chr>,<dbl>,<dbl>
AAACCCAAGCTTACGT-1,HC_01,3511,1257,Healthy,HC_01,10.48134,22.927941
AAACCCACAATTCACG-1,HC_01,8739,2486,Healthy,HC_01,14.52111,14.738528
AAACCCACACGCACCA-1,HC_01,6674,1907,Healthy,HC_01,11.59724,27.075217
AAACCCACATCTGGGC-1,HC_01,2082,911,Healthy,HC_01,36.50336,1.873199
AAACCCAGTAAGGCCA-1,HC_01,4093,1339,Healthy,HC_01,8.23357,28.267774
AAACCCAGTGGAAATT-1,HC_01,3933,1354,Healthy,HC_01,18.73888,24.332571


| Metric         | What it really is                  | Matrix operation                       |
| -------------- | ---------------------------------- | -------------------------------------- |
| `nCount_RNA`   | Total RNA counts per cell          | **Column sum**                         |
| `nFeature_RNA` | Number of expressed genes per cell | **Count of non-zero rows in a column** |


In [18]:
# Generate QC statistics table
qc_stats_before <- data.frame(
  Sample = names(seurat_list),
  Total_Cells = sapply(seurat_list, ncol),
  Median_Genes = sapply(seurat_list, function(x) median(x$nFeature_RNA)),
  Median_UMI = sapply(seurat_list, function(x) median(x$nCount_RNA)),
  Median_MT_percent = sapply(seurat_list, function(x) median(x$percent.mt))
)

write.csv(qc_stats_before, "results/QC/QC_stats_before_filtering.csv", row.names = FALSE)
print(qc_stats_before)

    Sample Total_Cells Median_Genes Median_UMI Median_MT_percent
HC1    HC1        8865         1413     4395.0          9.727512
HC2    HC2        7684         1603     4463.5          5.838465
HC3    HC3        7667         1478     4224.0          6.434316


In [19]:
# ============================================================================
# PART 3: QUALITY FILTERING
# ============================================================================

# Define QC thresholds
# These are standard thresholds - adjust based on your data
min_features <- 200
max_features <- 5000  # Adjust based on distribution
max_mt_percent <- 10  # 10% mitochondrial content
min_counts <- 500

# Filter cells
seurat_list_filtered <- lapply(seurat_list, function(x) {
  subset(
    x,
    subset = nFeature_RNA > min_features & 
             nFeature_RNA < max_features &
             nCount_RNA > min_counts &
             percent.mt < max_mt_percent
  )
})


In [20]:
# Visualize QC metrics after filtering
pdf("results/QC/02_QC_after_filtering.pdf", width = 15, height = 10)

for (i in names(seurat_list_filtered)) {
  p1 <- VlnPlot(
    seurat_list_filtered[[i]], 
    features = c("nFeature_RNA", "nCount_RNA", "percent.mt"),
    ncol = 3,
    pt.size = 0.1
  ) + plot_annotation(title = paste("Sample:", i, "(Filtered)"))
  print(p1)
}

dev.off()

Warning message:
“Default search for "data" layer in "RNA" assay yielded no results; utilizing "counts" layer instead.”


Warning message:
“Default search for "data" layer in "RNA" assay yielded no results; utilizing "counts" layer instead.”
Warning message:
“Default search for "data" layer in "RNA" assay yielded no results; utilizing "counts" layer instead.”


agg_record_312928385 
                   2

In [21]:
# QC statistics after filtering
qc_stats_after <- data.frame(
  Sample = names(seurat_list_filtered),
  Cells_After_Filtering = sapply(seurat_list_filtered, ncol),
  Cells_Removed = qc_stats_before$Total_Cells - sapply(seurat_list_filtered, ncol),
  Percent_Retained = round(
    sapply(seurat_list_filtered, ncol) / qc_stats_before$Total_Cells * 100, 2
  )
)

write.csv(qc_stats_after, "results/QC/QC_stats_after_filtering.csv", row.names = FALSE)
print(qc_stats_after)

    Sample Cells_After_Filtering Cells_Removed Percent_Retained
HC1    HC1                  4696          4169            52.97
HC2    HC2                  6059          1625            78.85
HC3    HC3                  6581          1086            85.84


In [22]:

# ============================================================================
# PART 4: NORMALIZATION AND IDENTIFICATION OF VARIABLE FEATURES
# ============================================================================

# Normalize data
seurat_list_filtered <- lapply(seurat_list_filtered, function(x) {
  x <- NormalizeData(x, normalization.method = "LogNormalize", scale.factor = 10000)
  x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 2000)
  return(x)
})

# Plot variable features
pdf("results/QC/03_variable_features.pdf", width = 12, height = 8)

for (i in names(seurat_list_filtered)) {
  top10 <- head(VariableFeatures(seurat_list_filtered[[i]]), 10)
  
  p1 <- VariableFeaturePlot(seurat_list_filtered[[i]])
  p2 <- LabelPoints(plot = p1, points = top10, repel = TRUE, xnudge = 0, ynudge = 0)
  
  print(p2 + ggtitle(paste("Top Variable Genes -", i)))
}

dev.off()


Normalizing layer: counts



Finding variable features for layer counts

Normalizing layer: counts

Finding variable features for layer counts

Normalizing layer: counts

Finding variable features for layer counts

Warning message in scale_x_log10():
“log-10 transformation introduced infinite values.”
Warning message in scale_x_log10():
“log-10 transformation introduced infinite values.”
Warning message in scale_x_log10():
“log-10 transformation introduced infinite values.”


agg_record_941905308 
                   2